In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
from functools import reduce
pd.options.mode.chained_assignment = None  # default='warn'
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [43]:
data = pd.read_csv('../data/replica _spring2023weekday_people.csv')

In [44]:
data.head()

,household_id,household_income,home_cty
0,3905272095708225430,0,"Sumner County, TN"
1,13433200847466215497,0,"Sumner County, TN"
2,12078924630778616133,0,"Sumner County, TN"
3,15618061633191836346,0,"Sumner County, TN"
4,13191095857763999111,0,"Sumner County, TN"


In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2059441 entries, 0 to 2059440
Data columns (total 3 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   household_id      uint64
 1   household_income  int64 
 2   home_cty          object
dtypes: int64(1), object(1), uint64(1)
memory usage: 47.1+ MB


In [46]:
data = data.drop_duplicates().reset_index(drop = True)

In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 826366 entries, 0 to 826365
Data columns (total 3 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   household_id      826366 non-null  uint64
 1   household_income  826366 non-null  int64 
 2   home_cty          826366 non-null  object
dtypes: int64(1), object(1), uint64(1)
memory usage: 18.9+ MB


In [47]:
data.head()

,household_id,household_income,home_cty
0,3905272095708225430,0,"Sumner County, TN"
1,13433200847466215497,0,"Sumner County, TN"
2,12078924630778616133,0,"Sumner County, TN"
3,15618061633191836346,0,"Sumner County, TN"
4,13191095857763999111,0,"Sumner County, TN"


In [48]:
data['Less than 200% FPL'] = np.where(data['household_income'] < 43920, 1, 0)

In [49]:
data.tail()

,household_id,household_income,home_cty,Less than 200% FPL
826361,13541919717057648004,37887,"Williamson County, TN",1
826362,7963926726522953891,37887,"Williamson County, TN",1
826363,13012792488368306290,52479,"Williamson County, TN",0
826364,9258347689406928808,52479,"Williamson County, TN",0
826365,6774732621778024471,52479,"Williamson County, TN",0


In [50]:
data['More than 300% FPL'] = np.where(data['household_income'] > 65880, 1, 0)

In [51]:
households = data.groupby('home_cty')['household_id'].nunique().reset_index()

In [52]:
households.columns = ['home_cty', 'unique_hhs']

In [53]:
households.head()

,home_cty,unique_hhs
0,"Cheatham County, TN",15667
1,"Davidson County, TN",317925
2,"Dickson County, TN",20888
3,"Houston County, TN",3217
4,"Humphreys County, TN",7265


In [54]:
df1 = data.groupby('home_cty')['Less than 200% FPL'].sum().reset_index()

In [55]:
df1.head()

,home_cty,Less than 200% FPL
0,"Cheatham County, TN",4699
1,"Davidson County, TN",117585
2,"Dickson County, TN",7528
3,"Houston County, TN",1692
4,"Humphreys County, TN",3239


In [56]:
df1 = df1.merge(households, on = 'home_cty')

In [32]:
df1

,home_cty,Less than 200% FPL,unique_hhs,% of All Households with HHIncome less than 200% FPL
0,"Cheatham County, TN",4699,15667,0.299930
1,"Davidson County, TN",117585,317925,0.369851
2,"Dickson County, TN",7528,20888,0.360398
3,"Houston County, TN",1692,3217,0.525956
4,"Humphreys County, TN",3239,7265,0.445836
5,"Montgomery County, TN",29268,81867,0.357507
6,"Robertson County, TN",9148,27445,0.333321
7,"Rutherford County, TN",37017,125327,0.295363
8,"Stewart County, TN",2366,5479,0.431831
9,"Sumner County, TN",21872,74553,0.293375


In [57]:
df2 = data.groupby('home_cty')['More than 300% FPL'].sum().reset_index()

In [58]:
df2.head()

,home_cty,More than 300% FPL
0,"Cheatham County, TN",7906
1,"Davidson County, TN",150642
2,"Dickson County, TN",9333
3,"Houston County, TN",1033
4,"Humphreys County, TN",2609


In [59]:
df = df1.merge(df2, on = 'home_cty')

In [61]:
df = df.set_index('home_cty').transpose()

In [62]:
df.head()

home_cty,"Cheatham County, TN","Davidson County, TN","Dickson County, TN","Houston County, TN","Humphreys County, TN","Montgomery County, TN","Robertson County, TN","Rutherford County, TN","Stewart County, TN","Sumner County, TN","Trousdale County, TN","Williamson County, TN","Wilson County, TN"
Less than 200% FPL,4699,117585,7528,1692,3239,29268,9148,37017,2366,21872,3393,13573,15350
unique_hhs,15667,317925,20888,3217,7265,81867,27445,125327,5479,74553,5583,86245,54905
More than 300% FPL,7906,150642,9333,1033,2609,37804,13484,67790,2148,40884,1261,64172,32387


In [63]:
thelist = [df['Cheatham County, TN'], df['Davidson County, TN'], df['Dickson County, TN'], df['Houston County, TN'], df['Humphreys County, TN'], 
           df['Montgomery County, TN'], df['Robertson County, TN'], df['Rutherford County, TN'], df['Stewart County, TN'], df['Sumner County, TN'], 
           df['Trousdale County, TN'], df['Williamson County, TN'], df['Wilson County, TN']]
df['GNRC'] = sum(thelist)

In [65]:
df = df.transpose().reset_index(drop = False)

In [69]:
df.tail()

,home_cty,Less than 200% FPL,unique_hhs,More than 300% FPL,% of All Households with HHIncome less than 200% FPL,% of All Households with HHIncome more than 300% FPL
9,"Sumner County, TN",21872,74553,40884,0.293375,0.548388
10,"Trousdale County, TN",3393,5583,1261,0.607738,0.225864
11,"Williamson County, TN",13573,86245,64172,0.157377,0.744066
12,"Wilson County, TN",15350,54905,32387,0.279574,0.589873
13,GNRC,266730,826366,431453,NaN,NaN


In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 6 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   home_cty                                              14 non-null     object 
 1   Less than 200% FPL                                    14 non-null     int64  
 2   unique_hhs                                            14 non-null     int64  
 3   More than 300% FPL                                    14 non-null     int64  
 4   % of All Households with HHIncome less than 200% FPL  13 non-null     float64
 5   % of All Households with HHIncome more than 300% FPL  13 non-null     float64
dtypes: float64(2), int64(3), object(1)
memory usage: 804.0+ bytes


In [73]:
df['% of All Households with HHIncome less than 200% FPL'] = df['Less than 200% FPL']/df['unique_hhs']
df['% of All Households with HHIncome more than 300% FPL'] = df['More than 300% FPL']/df['unique_hhs']

In [74]:
df

,home_cty,Less than 200% FPL,unique_hhs,More than 300% FPL,% of All Households with HHIncome less than 200% FPL,% of All Households with HHIncome more than 300% FPL
0,"Cheatham County, TN",4699,15667,7906,0.299930,0.504628
1,"Davidson County, TN",117585,317925,150642,0.369851,0.473829
2,"Dickson County, TN",7528,20888,9333,0.360398,0.446812
3,"Houston County, TN",1692,3217,1033,0.525956,0.321107
4,"Humphreys County, TN",3239,7265,2609,0.445836,0.359119
5,"Montgomery County, TN",29268,81867,37804,0.357507,0.461773
6,"Robertson County, TN",9148,27445,13484,0.333321,0.491310
7,"Rutherford County, TN",37017,125327,67790,0.295363,0.540905
8,"Stewart County, TN",2366,5479,2148,0.431831,0.392042
9,"Sumner County, TN",21872,74553,40884,0.293375,0.548388


In [75]:
df.to_csv('../data/output.csv', index = False)